# Multi-Agent Autonomous Research Assistant

This notebook implements a multi-agent autonomous research system using LangGraph/LangChain. The system automates the academic research process by:

1. Accepting user-provided research keywords
2. Retrieving relevant academic papers from multiple sources
3. Ranking and selecting the most significant papers
4. Summarizing their contents using a fine-tuned model
5. Comparing insights to identify common themes, contradictions, and research gaps

The system consists of five specialized agents orchestrated using LangGraph:

1. **KeywordAgent**: Enhances the user's input by generating expanded and related keywords
2. **SearchAgent**: Interfaces with academic search APIs to retrieve papers
3. **RankAgent**: Scores and ranks papers using a multi-criteria strategy
4. **SummaryAgent**: Processes selected papers using a fine-tuned summarization model
5. **CompareAgent**: Performs comparative analysis on the summarized content

## Table of Contents
1. [Environment Setup](#1-environment-setup)
2. [Agent Implementation](#2-agent-implementation)
3. [Agent Orchestration with LangGraph](#3-agent-orchestration-with-langgraph)
4. [Testing and Evaluation](#4-testing-and-evaluation)
5. [Research Report Generation](#5-research-report-generation)

## 1. Environment Setup

First, let's install all the required libraries and dependencies.

In [ ]:
# Install required libraries
!pip install langchain langchain_openai "langgraph>=0.0.19" pydantic langchain_community langchain_core bs4 scholarly arxiv feedparser tqdm nltk matplotlib pandas numpy requests torch transformers rich lxml streamlit together

In [ ]:
# Import required libraries
import os
import json
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from typing import List, Dict, Any, Optional, Tuple, Union, Callable
from pydantic import BaseModel, Field
from tqdm import tqdm
import requests
import re
import json
from datetime import datetime
from bs4 import BeautifulSoup
import arxiv
import feedparser
import xml.etree.ElementTree as ET
from urllib.parse import quote
import nltk
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords

# Import LangChain and LangGraph
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain.output_parsers import PydanticOutputParser
from langchain_core.messages import HumanMessage, AIMessage

# Import LangGraph components
from langgraph.graph import END, StateGraph
from langgraph.graph.message import add_messages, get_messages_by_id, MessageGraph
from langgraph.prebuilt import create_agent_executor

# Import LLM components
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain

# Import LangChain community components
from langchain_community.tools import BaseTool
from langchain_community.utilities import SerpAPIWrapper

# Import Transformers for the fine-tuned model
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from peft import PeftModel

# Download NLTK data
nltk.download('punkt')
nltk.download('stopwords')

# Set up API keys
# Replace with your actual API keys
api_keys = {
    "TOGETHER_API_KEY": "YOUR_TOGETHER_API_KEY",
    "OPENAI_API_KEY": "YOUR_OPENAI_API_KEY",
    "SEMANTIC_SCHOLAR_API_KEY": "YOUR_SEMANTIC_SCHOLAR_API_KEY" # Optional
}

# Set environment variables
for key, value in api_keys.items():
    os.environ[key] = value

# Initialize Together.ai API
import together
together.api_key = os.environ.get("TOGETHER_API_KEY")

# Initialize LLM
llm = ChatOpenAI(
    model="gpt-3.5-turbo",  # You can replace with a different model
    temperature=0.3,
    max_tokens=1024
)

# Alternative: Use Together.ai API
# from langchain_community.llms import Together
# llm = Together(
#     model="meta-llama/Llama-3-8B-Instruct",
#     temperature=0.3,
#     max_tokens=1024
# )

### 1.1 Load the Fine-Tuned Summarization Model

Load the fine-tuned model from Part A for paper summarization.

In [ ]:
# Load the fine-tuned model
def load_fine_tuned_model(model_path="./fine_tuned_model"):
    """
    Load the fine-tuned model for paper summarization.
    
    Args:
        model_path: Path to the fine-tuned model
        
    Returns:
        model, tokenizer: The loaded model and tokenizer
    """
    # Check if model path exists
    if not os.path.exists(model_path):
        print(f"Warning: Model path {model_path} does not exist.")
        print("Using fallback method with Together.ai API for summarization.")
        return None, None
    
    try:
        # Load the base model
        base_model_name = "mistralai/Mistral-7B-v0.1"  # Use the same base model as in Part A
        
        # Load tokenizer
        tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
        tokenizer.pad_token = tokenizer.eos_token
        
        # Check if CUDA is available
        device = "cuda" if torch.cuda.is_available() else "cpu"
        print(f"Using device: {device}")
        
        # Load the base model
        base_model = AutoModelForCausalLM.from_pretrained(
            base_model_name,
            load_in_4bit=True if torch.cuda.is_available() else False,
            torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
            device_map="auto" if torch.cuda.is_available() else None,
            trust_remote_code=True
        )
        
        # Load the fine-tuned model
        model = PeftModel.from_pretrained(base_model, model_path)
        model.eval()  # Set model to evaluation mode
        
        print("Fine-tuned model loaded successfully.")
        return model, tokenizer
    except Exception as e:
        print(f"Error loading fine-tuned model: {e}")
        print("Using fallback method with Together.ai API for summarization.")
        return None, None

# Load the fine-tuned model
model, tokenizer = load_fine_tuned_model()

### 1.2 Define Data Models

Define Pydantic models for the data structures used in the multi-agent system.

In [ ]:
# Define data models
class Keyword(BaseModel):
    """A keyword or phrase for academic search."""
    keyword: str = Field(description="The keyword or phrase")
    relevance: Optional[float] = Field(default=1.0, description="Relevance score from 0.0 to 1.0")
    
class KeywordSet(BaseModel):
    """A set of keywords for academic search."""
    original_query: str = Field(description="Original user query")
    expanded_keywords: List[Keyword] = Field(description="List of expanded keywords")
    
class Author(BaseModel):
    """An author of an academic paper."""
    name: str = Field(description="Author's name")
    affiliation: Optional[str] = Field(default=None, description="Author's affiliation")
    
class Paper(BaseModel):
    """An academic paper."""
    id: str = Field(description="Unique identifier for the paper")
    title: str = Field(description="Title of the paper")
    authors: List[Author] = Field(description="List of authors")
    abstract: str = Field(description="Abstract of the paper")
    year: Optional[int] = Field(default=None, description="Year of publication")
    url: Optional[str] = Field(default=None, description="URL to the paper")
    pdf_url: Optional[str] = Field(default=None, description="URL to the PDF")
    citation_count: Optional[int] = Field(default=None, description="Number of citations")
    source: str = Field(description="Source of the paper (e.g., 'arxiv', 'semantic_scholar')")
    keywords: Optional[List[str]] = Field(default=None, description="Keywords associated with the paper")
    full_text: Optional[str] = Field(default=None, description="Full text of the paper if available")
    
class RankedPaper(Paper):
    """An academic paper with ranking information."""
    rank_score: float = Field(description="Overall ranking score")
    relevance_score: float = Field(description="Relevance to the query")
    citation_score: float = Field(description="Score based on citations")
    recency_score: float = Field(description="Score based on publication recency")
    
class PaperSummary(BaseModel):
    """A summary of an academic paper."""
    paper_id: str = Field(description="ID of the summarized paper")
    title: str = Field(description="Title of the paper")
    authors: List[str] = Field(description="List of author names")
    summary: str = Field(description="Generated summary of the paper")
    methodology: Optional[str] = Field(default=None, description="Summary of the methodology used")
    contributions: Optional[str] = Field(default=None, description="Key contributions of the paper")
    limitations: Optional[str] = Field(default=None, description="Limitations or gaps identified")
    
class ComparativeAnalysis(BaseModel):
    """Comparative analysis of multiple paper summaries."""
    common_findings: List[str] = Field(description="Common findings across papers")
    contradictions: List[str] = Field(description="Contradictory findings between papers")
    research_gaps: List[str] = Field(description="Identified research gaps")
    future_directions: List[str] = Field(description="Suggested future research directions")
    
class ResearchReport(BaseModel):
    """Complete research report."""
    topic: str = Field(description="Research topic")
    topic_summary: str = Field(description="Brief overview of the topic")
    expanded_keywords: List[str] = Field(description="Expanded keywords used for search")
    paper_summaries: List[PaperSummary] = Field(description="Summaries of top papers")
    comparative_analysis: ComparativeAnalysis = Field(description="Comparative analysis of papers")
    timestamp: str = Field(description="Timestamp of the report generation")

## 2. Agent Implementation

Implement each of the five specialized agents.

### 2.1 KeywordAgent

Enhances the user's input by generating expanded and related keywords.

In [ ]:
class KeywordAgent:
    """Agent that expands user-provided keywords into a comprehensive set of search terms."""
    
    def __init__(self, llm):
        self.llm = llm
        
        # Define the prompt template
        self.prompt_template = PromptTemplate(
            input_variables=["query"],
            template="""You are an expert research assistant specialized in expanding search queries into comprehensive sets of keywords for academic research.

Given the following research query, generate an expanded list of 8-12 related keywords and phrases that would be useful for finding relevant academic papers. 
Include both broader and more specific terms. Consider synonyms, related concepts, and any important subtopics.

Research query: {query}

Please respond with a JSON object in the following format:
{{"original_query": "the original query",
"expanded_keywords": [
    {{"keyword": "first keyword", "relevance": 0.9}},
    {{"keyword": "second keyword", "relevance": 0.8}},
    ... and so on
]}}

Relevance should be a float between 0.0 and 1.0, where 1.0 indicates highest relevance to the original query.
Order the keywords by relevance, with the most relevant ones first.
"""
        )
        
        # Set up the output parser
        self.output_parser = JsonOutputParser()
        
        # Create the chain
        self.chain = self.prompt_template | self.llm | self.output_parser
    
    def process(self, query: str) -> KeywordSet:
        """Expand the user's query into a set of related keywords."""
        try:
            # Generate expanded keywords using the LLM
            result = self.chain.invoke({"query": query})
            
            # Convert the result to a KeywordSet object
            expanded_keywords = [Keyword(keyword=item["keyword"], relevance=item["relevance"]) 
                                for item in result["expanded_keywords"]]
            
            return KeywordSet(
                original_query=result["original_query"],
                expanded_keywords=expanded_keywords
            )
        except Exception as e:
            print(f"Error in KeywordAgent: {e}")
            # Fallback: return the original query as a keyword
            return KeywordSet(
                original_query=query,
                expanded_keywords=[Keyword(keyword=query, relevance=1.0)]
            )
    
    def __call__(self, query: str) -> KeywordSet:
        """Process the query and return a KeywordSet."""
        return self.process(query)

### 2.2 SearchAgent

Interfaces with academic search APIs to retrieve relevant papers.

In [ ]:
class SearchAgent:
    """Agent that searches for academic papers across multiple sources."""
    
    def __init__(self):
        self.MAX_RESULTS_PER_SOURCE = 5  # Max results to retrieve from each source
        self.semantic_scholar_api_key = os.environ.get("SEMANTIC_SCHOLAR_API_KEY")
    
    def search_arxiv(self, query, max_results=5):
        """Search arXiv for papers matching the query."""
        try:
            # Use the arXiv API via the arxiv library
            search = arxiv.Search(
                query=query,
                max_results=max_results,
                sort_by=arxiv.SortCriterion.Relevance
            )
            
            papers = []
            for result in search.results():
                # Extract the year from the published date
                year = result.published.year if hasattr(result, 'published') else None
                
                paper = Paper(
                    id=result.entry_id.split('/')[-1],
                    title=result.title,
                    authors=[Author(name=author.name) for author in result.authors],
                    abstract=result.summary,
                    year=year,
                    url=result.entry_id,
                    pdf_url=result.pdf_url,
                    citation_count=None,  # arXiv doesn't provide citation count
                    source="arxiv",
                    keywords=None,  # arXiv doesn't provide keywords
                )
                papers.append(paper)
            
            return papers
        except Exception as e:
            print(f"Error searching arXiv: {e}")
            return []
    
    def search_arxiv_api(self, query, max_results=5):
        """Search arXiv using the official API."""
        try:
            # Format the query for the arXiv API
            encoded_query = quote(query)
            url = f"http://export.arxiv.org/api/query?search_query=all:{encoded_query}&start=0&max_results={max_results}"
            
            # Send the request
            response = requests.get(url)
            response.raise_for_status()
            
            # Parse the response using feedparser
            feed = feedparser.parse(response.content)
            
            papers = []
            for entry in feed.entries:
                # Extract the year from the published date
                published = entry.get('published', '')
                year = int(published[:4]) if published and len(published) >= 4 else None
                
                # Extract authors
                authors = []
                for author in entry.get('authors', []):
                    name = author.get('name', '')
                    if name:
                        authors.append(Author(name=name))
                
                # Create the paper object
                paper = Paper(
                    id=entry.get('id', '').split('/')[-1],
                    title=entry.get('title', ''),
                    authors=authors,
                    abstract=entry.get('summary', ''),
                    year=year,
                    url=entry.get('link', ''),
                    pdf_url=next((link.href for link in entry.get('links', []) if link.get('title') == 'pdf'), None),
                    citation_count=None,
                    source="arxiv",
                    keywords=None,
                )
                papers.append(paper)
            
            return papers
        except Exception as e:
            print(f"Error searching arXiv API: {e}")
            return []
    
    def search_semantic_scholar(self, query, max_results=5):
        """Search Semantic Scholar for papers matching the query."""
        try:
            # Use the Semantic Scholar API
            url = "https://api.semanticscholar.org/graph/v1/paper/search"
            params = {
                "query": query,
                "limit": max_results,
                "fields": "title,authors,abstract,year,url,citationCount,influentialCitationCount"
            }
            
            headers = {}
            if self.semantic_scholar_api_key:
                headers["x-api-key"] = self.semantic_scholar_api_key
            
            response = requests.get(url, params=params, headers=headers)
            response.raise_for_status()
            data = response.json()
            
            papers = []
            for item in data.get('data', []):
                # Extract authors
                authors = []
                for author in item.get('authors', []):
                    name = author.get('name', '')
                    if name:
                        authors.append(Author(name=name))
                
                # Create the paper object
                paper = Paper(
                    id=item.get('paperId', ''),
                    title=item.get('title', ''),
                    authors=authors,
                    abstract=item.get('abstract', ''),
                    year=item.get('year'),
                    url=item.get('url'),
                    pdf_url=None,  # Semantic Scholar API doesn't directly provide PDF URL
                    citation_count=item.get('citationCount'),
                    source="semantic_scholar",
                    keywords=None,
                )
                papers.append(paper)
            
            return papers
        except Exception as e:
            print(f"Error searching Semantic Scholar: {e}")
            return []
    
    def search_pubmed(self, query, max_results=5):
        """Search PubMed for papers matching the query."""
        try:
            # Format the query for the PubMed eUtils API
            encoded_query = quote(query)
            
            # First get the IDs of matching articles
            esearch_url = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term={encoded_query}&retmode=json&retmax={max_results}"
            esearch_response = requests.get(esearch_url)
            esearch_response.raise_for_status()
            esearch_data = esearch_response.json()
            
            # Get the list of IDs
            id_list = esearch_data.get('esearchresult', {}).get('idlist', [])
            if not id_list:
                return []
            
            # Then fetch details for these IDs
            ids_string = ",".join(id_list)
            efetch_url = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id={ids_string}&retmode=xml"
            efetch_response = requests.get(efetch_url)
            efetch_response.raise_for_status()
            
            # Parse the XML response
            root = ET.fromstring(efetch_response.content)
            papers = []
            
            for article in root.findall(".//PubmedArticle"):
                try:
                    # Extract PubMed ID
                    pmid = article.find(".//PMID").text if article.find(".//PMID") is not None else "unknown"
                    
                    # Extract article title
                    title_element = article.find(".//ArticleTitle")
                    title = title_element.text if title_element is not None else "Unknown Title"
                    
                    # Extract abstract
                    abstract_elements = article.findall(".//AbstractText")
                    abstract = " ".join([elem.text for elem in abstract_elements if elem.text is not None]) if abstract_elements else ""
                    
                    # Extract year
                    year_element = article.find(".//PubDate/Year")
                    year = int(year_element.text) if year_element is not None and year_element.text.isdigit() else None
                    
                    # Extract authors
                    authors = []
                    author_elements = article.findall(".//Author")
                    for author_elem in author_elements:
                        last_name = author_elem.find("LastName")
                        fore_name = author_elem.find("ForeName")
                        name = ""
                        if last_name is not None and last_name.text is not None:
                            name += last_name.text
                            if fore_name is not None and fore_name.text is not None:
                                name = f"{fore_name.text} {name}"
                        if name:
                            authors.append(Author(name=name))
                    
                    # Create URL to the PubMed article
                    url = f"https://pubmed.ncbi.nlm.nih.gov/{pmid}/"
                    
                    # Create paper object
                    paper = Paper(
                        id=pmid,
                        title=title,
                        authors=authors,
                        abstract=abstract,
                        year=year,
                        url=url,
                        pdf_url=None,  # PubMed doesn't directly provide PDF URLs
                        citation_count=None,  # PubMed doesn't provide citation counts
                        source="pubmed",
                        keywords=None,  # We could extract keywords if needed
                    )
                    papers.append(paper)
                except Exception as e:
                    print(f"Error processing PubMed article: {e}")
                    continue
            
            return papers
        except Exception as e:
            print(f"Error searching PubMed: {e}")
            return []
    
    def process(self, keyword_set: KeywordSet) -> List[Paper]:
        """Search for papers using the expanded keywords."""
        all_papers = []
        
        # Use the top 5 most relevant keywords for searching
        top_keywords = sorted(keyword_set.expanded_keywords, key=lambda k: k.relevance, reverse=True)[:5]
        
        # Search using the original query first
        print(f"Searching for papers with query: {keyword_set.original_query}")
        
        # Search arXiv
        arxiv_papers = self.search_arxiv_api(keyword_set.original_query, self.MAX_RESULTS_PER_SOURCE)
        if arxiv_papers:
            print(f"Found {len(arxiv_papers)} papers from arXiv")
            all_papers.extend(arxiv_papers)
        
        # Search PubMed
        pubmed_papers = self.search_pubmed(keyword_set.original_query, self.MAX_RESULTS_PER_SOURCE)
        if pubmed_papers:
            print(f"Found {len(pubmed_papers)} papers from PubMed")
            all_papers.extend(pubmed_papers)
        
        # Search Semantic Scholar if API key is available
        if self.semantic_scholar_api_key:
            ss_papers = self.search_semantic_scholar(keyword_set.original_query, self.MAX_RESULTS_PER_SOURCE)
            if ss_papers:
                print(f"Found {len(ss_papers)} papers from Semantic Scholar")
                all_papers.extend(ss_papers)
        
        # Search using each of the top keywords
        for keyword in top_keywords:
            print(f"Searching for papers with keyword: {keyword.keyword}")
            
            # Search arXiv
            arxiv_papers = self.search_arxiv_api(keyword.keyword, self.MAX_RESULTS_PER_SOURCE)
            if arxiv_papers:
                print(f"Found {len(arxiv_papers)} papers from arXiv for keyword: {keyword.keyword}")
                all_papers.extend(arxiv_papers)
            
            # Search PubMed
            pubmed_papers = self.search_pubmed(keyword.keyword, self.MAX_RESULTS_PER_SOURCE)
            if pubmed_papers:
                print(f"Found {len(pubmed_papers)} papers from PubMed for keyword: {keyword.keyword}")
                all_papers.extend(pubmed_papers)
            
            # Search Semantic Scholar if API key is available
            if self.semantic_scholar_api_key:
                ss_papers = self.search_semantic_scholar(keyword.keyword, self.MAX_RESULTS_PER_SOURCE)
                if ss_papers:
                    print(f"Found {len(ss_papers)} papers from Semantic Scholar for keyword: {keyword.keyword}")
                    all_papers.extend(ss_papers)
        
        # Remove duplicates based on paper ID
        unique_papers = []
        paper_ids = set()
        
        for paper in all_papers:
            if paper.id not in paper_ids:
                unique_papers.append(paper)
                paper_ids.add(paper.id)
        
        print(f"Total unique papers found: {len(unique_papers)}")
        return unique_papers
    
    def __call__(self, keyword_set: KeywordSet) -> List[Paper]:
        """Process the keyword set and return a list of papers."""
        return self.process(keyword_set)

### 2.3 RankAgent

Scores and ranks papers using a multi-criteria strategy.

In [ ]:
class RankAgent:
    """Agent that ranks academic papers based on multiple criteria."""
    
    def __init__(self, llm):
        self.llm = llm
        self.current_year = datetime.now().year
        
        # Define the prompt template for relevance scoring
        self.relevance_prompt_template = PromptTemplate(
            input_variables=["query", "title", "abstract"],
            template="""You are an expert academic researcher evaluating the relevance of a paper to a research query.

Research query: {query}

Paper title: {title}

Paper abstract: {abstract}

On a scale of 0.0 to 1.0, how relevant is this paper to the research query?
- 0.0: Not relevant at all
- 0.3: Somewhat relevant but only tangentially related
- 0.5: Moderately relevant
- 0.7: Highly relevant but not perfect match
- 1.0: Extremely relevant, perfect match

First explain your reasoning in 2-3 sentences, then provide a single number between 0.0 and 1.0 as your final relevance score.
Your response should end with a line containing only the number as a floating point value.
"""
        )
        
        # Set up the chain for relevance scoring
        self.relevance_chain = self.relevance_prompt_template | self.llm | StrOutputParser()
    
    def calculate_citation_score(self, citation_count):
        """Calculate a score based on citation count."""
        if citation_count is None:
            return 0.5  # Default score when citation count is unknown
        
        # Log-based scoring to handle papers with very high citation counts
        if citation_count == 0:
            return 0.1
        return min(1.0, 0.1 + 0.3 * np.log10(citation_count + 1))
    
    def calculate_recency_score(self, year):
        """Calculate a score based on publication recency."""
        if year is None:
            return 0.5  # Default score when year is unknown
        
        # Linear decay from current year (1.0) to 10 years ago (0.1)
        years_ago = self.current_year - year
        if years_ago <= 0:
            return 1.0  # Current or future publication (unlikely, but possible)
        elif years_ago >= 10:
            return 0.1  # 10+ years old
        else:
            return 1.0 - (0.9 * years_ago / 10.0)
    
    def calculate_relevance_score(self, paper, query):
        """Calculate a relevance score using the LLM."""
        try:
            # Generate relevance analysis using the LLM
            result = self.relevance_chain.invoke({
                "query": query,
                "title": paper.title,
                "abstract": paper.abstract
            })
            
            # Extract the relevance score (last line of the result)
            lines = result.strip().split('\n')
            score_text = lines[-1].strip()
            
            # Extract the floating point value
            score_match = re.search(r'\d+\.\d+|\d+', score_text)
            if score_match:
                score = float(score_match.group())
                return min(1.0, max(0.0, score))  # Clamp between 0.0 and 1.0
            
            # Fallback: use a default score
            return 0.5
        except Exception as e:
            print(f"Error calculating relevance score: {e}")
            return 0.5  # Default score on error
    
    def calculate_rank_score(self, relevance_score, citation_score, recency_score):
        """Calculate an overall rank score by combining the individual scores."""
        # Weights for each score (can be adjusted)
        weights = {
            "relevance": 0.6,  # Relevance is most important
            "citation": 0.25,  # Citations indicate impact
            "recency": 0.15    # Recency ensures up-to-date information
        }
        
        return (
            weights["relevance"] * relevance_score +
            weights["citation"] * citation_score +
            weights["recency"] * recency_score
        )
    
    def process(self, papers: List[Paper], keyword_set: KeywordSet) -> List[RankedPaper]:
        """Rank the papers based on multiple criteria."""
        ranked_papers = []
        
        print("Ranking papers...")
        for i, paper in enumerate(papers):
            print(f"Ranking paper {i+1}/{len(papers)}: {paper.title}")
            
            # Calculate individual scores
            relevance_score = self.calculate_relevance_score(paper, keyword_set.original_query)
            citation_score = self.calculate_citation_score(paper.citation_count)
            recency_score = self.calculate_recency_score(paper.year)
            
            # Calculate overall rank score
            rank_score = self.calculate_rank_score(relevance_score, citation_score, recency_score)
            
            # Create a RankedPaper object
            ranked_paper = RankedPaper(
                **paper.dict(),
                rank_score=rank_score,
                relevance_score=relevance_score,
                citation_score=citation_score,
                recency_score=recency_score
            )
            
            ranked_papers.append(ranked_paper)
        
        # Sort papers by rank score in descending order
        ranked_papers.sort(key=lambda p: p.rank_score, reverse=True)
        
        return ranked_papers
    
    def __call__(self, papers: List[Paper], keyword_set: KeywordSet) -> List[RankedPaper]:
        """Process the papers and return a list of ranked papers."""
        return self.process(papers, keyword_set)

### 2.4 SummaryAgent

Processes selected top-ranked papers and generates structured summaries.

In [ ]:
class SummaryAgent:
    """Agent that generates summaries of academic papers."""
    
    def __init__(self, model=None, tokenizer=None, llm=None):
        self.model = model
        self.tokenizer = tokenizer
        self.llm = llm
        
        # Define the prompt template for structure extraction
        self.structure_prompt_template = PromptTemplate(
            input_variables=["summary"],
            template="""You are an expert academic researcher and summarizer. Extract and structure the information from the following paper summary into specific sections.

Paper summary: {summary}

Extract the following components from the summary:
1. Methodology: What research methods or techniques were used?
2. Key Contributions: What are the main findings or contributions of the paper?
3. Limitations or Gaps: What limitations, open questions, or research gaps were identified?

Respond with a JSON object in the following format:
{{"methodology": "description of methodology",
"contributions": "key contributions and findings",
"limitations": "limitations and research gaps"}}

Include only the JSON in your response, with no additional text.
"""
        )
        
        # Set up the output parser for structured information
        self.structure_parser = JsonOutputParser()
        
        # Create the chain for structure extraction
        self.structure_chain = self.structure_prompt_template | self.llm | self.structure_parser
        
        # Define the prompt template for paper summarization with Together.ai (fallback method)
        self.together_prompt_template = """Summarize the following academic paper with a focus on the main problem, methods, and key findings.
        
Title: {title}
Authors: {authors}
Abstract: {abstract}
        
Provide a concise but comprehensive summary of the paper that covers the research problem, methodology, and major contributions.
Summary:
"""
    
    def summarize_with_fine_tuned_model(self, paper):
        """Generate a summary using the fine-tuned model from Part A."""
        if self.model is None or self.tokenizer is None:
            return None
        
        try:
            # Create the prompt
            prompt = f"""Summarize the following academic paper:
            
Article: {paper.abstract}
            
Summary: """
            
            # Tokenize the input
            inputs = self.tokenizer(prompt, return_tensors="pt").to(self.model.device)
            
            # Generate the summary
            output = self.model.generate(
                inputs.input_ids,
                max_new_tokens=300,
                do_sample=True,
                temperature=0.7,
                top_p=0.9,
                repetition_penalty=1.2,
                pad_token_id=self.tokenizer.eos_token_id
            )
            
            # Decode the output
            summary = self.tokenizer.decode(output[0], skip_special_tokens=True)
            
            # Extract just the summary part (after "Summary:")
            try:
                summary = summary.split("Summary:")[1].strip()
            except IndexError:
                pass  # If "Summary:" is not in the output, use the whole output
            
            return summary
        except Exception as e:
            print(f"Error generating summary with fine-tuned model: {e}")
            return None
    
    def summarize_with_together(self, paper):
        """Generate a summary using the Together.ai API (fallback method)."""
        try:
            # Prepare the author string
            authors_str = ", ".join([author.name for author in paper.authors])
            
            # Create the prompt
            prompt = self.together_prompt_template.format(
                title=paper.title,
                authors=authors_str,
                abstract=paper.abstract
            )
            
            # Call the Together.ai API
            response = together.Complete.create(
                prompt=prompt,
                model="meta-llama/Llama-3-8B-Instruct",
                max_tokens=512,
                temperature=0.7,
                top_p=0.9,
                top_k=60,
                repetition_penalty=1.1
            )
            
            # Extract the summary
            summary = response['output']['choices'][0]['text']
            return summary
        except Exception as e:
            print(f"Error generating summary with Together.ai: {e}")
            return f"Unable to generate summary due to an error: {e}"
    
    def extract_structured_information(self, summary):
        """Extract structured information from the summary."""
        try:
            result = self.structure_chain.invoke({"summary": summary})
            return result
        except Exception as e:
            print(f"Error extracting structured information: {e}")
            return {
                "methodology": "Not available",
                "contributions": "Not available",
                "limitations": "Not available"
            }
    
    def process(self, ranked_papers: List[RankedPaper], top_n=3) -> List[PaperSummary]:
        """Generate summaries for the top-ranked papers."""
        paper_summaries = []
        
        # Select the top N papers
        top_papers = ranked_papers[:top_n]
        
        print(f"Generating summaries for top {len(top_papers)} papers...")
        for i, paper in enumerate(top_papers):
            print(f"Summarizing paper {i+1}/{len(top_papers)}: {paper.title}")
            
            # First try to summarize with the fine-tuned model
            summary = self.summarize_with_fine_tuned_model(paper)
            
            # If that fails, use Together.ai as a fallback
            if summary is None:
                print("Using Together.ai API for summarization (fallback method)")
                summary = self.summarize_with_together(paper)
            
            # Extract structured information from the summary
            structured_info = self.extract_structured_information(summary)
            
            # Create a PaperSummary object
            paper_summary = PaperSummary(
                paper_id=paper.id,
                title=paper.title,
                authors=[author.name for author in paper.authors],
                summary=summary,
                methodology=structured_info.get("methodology"),
                contributions=structured_info.get("contributions"),
                limitations=structured_info.get("limitations")
            )
            
            paper_summaries.append(paper_summary)
        
        return paper_summaries
    
    def __call__(self, ranked_papers: List[RankedPaper], top_n=3) -> List[PaperSummary]:
        """Process the ranked papers and return a list of paper summaries."""
        return self.process(ranked_papers, top_n)

### 2.5 CompareAgent

Performs comparative analysis on the summarized content.

In [ ]:
class CompareAgent:
    """Agent that performs comparative analysis on multiple paper summaries."""
    
    def __init__(self, llm):
        self.llm = llm
        
        # Define the prompt template for comparative analysis
        self.prompt_template = PromptTemplate(
            input_variables=["summaries"],
            template="""You are an expert academic researcher analyzing multiple papers on a related topic. Perform a comparative analysis of the following paper summaries to identify common themes, contradictions, research gaps, and future directions.

Paper Summaries:
{summaries}

Based on these summaries, provide a comprehensive comparative analysis in the following JSON format:
{{"common_findings": [
    "Common finding 1",
    "Common finding 2",
    ... (at least 3-5 items)
],
"contradictions": [
    "Contradiction 1",
    "Contradiction 2",
    ... (identify any contradictions between papers)
],
"research_gaps": [
    "Research gap 1",
    "Research gap 2",
    ... (identify areas that need further research, at least 3-4 items)
],
"future_directions": [
    "Future direction 1",
    "Future direction 2",
    ... (suggest future research directions, at least 3-4 items)
]}}

Include only the JSON in your response, with no additional text.
"""
        )
        
        # Set up the output parser
        self.output_parser = JsonOutputParser()
        
        # Create the chain
        self.chain = self.prompt_template | self.llm | self.output_parser
    
    def format_paper_summaries(self, paper_summaries: List[PaperSummary]) -> str:
        """Format the paper summaries for the prompt."""
        formatted_summaries = ""
        
        for i, summary in enumerate(paper_summaries):
            formatted_summaries += f"Paper {i+1}: {summary.title}\n"
            formatted_summaries += f"Authors: {', '.join(summary.authors)}\n"
            formatted_summaries += f"Summary: {summary.summary}\n"
            formatted_summaries += f"Methodology: {summary.methodology}\n"
            formatted_summaries += f"Key Contributions: {summary.contributions}\n"
            formatted_summaries += f"Limitations/Gaps: {summary.limitations}\n\n"
        
        return formatted_summaries
    
    def process(self, paper_summaries: List[PaperSummary]) -> ComparativeAnalysis:
        """Perform comparative analysis on the paper summaries."""
        # Format the paper summaries for the prompt
        formatted_summaries = self.format_paper_summaries(paper_summaries)
        
        print("Performing comparative analysis...")
        try:
            # Generate comparative analysis using the LLM
            result = self.chain.invoke({"summaries": formatted_summaries})
            
            # Create a ComparativeAnalysis object
            comparative_analysis = ComparativeAnalysis(
                common_findings=result.get("common_findings", []),
                contradictions=result.get("contradictions", []),
                research_gaps=result.get("research_gaps", []),
                future_directions=result.get("future_directions", [])
            )
            
            return comparative_analysis
        except Exception as e:
            print(f"Error in comparative analysis: {e}")
            # Return a default ComparativeAnalysis object
            return ComparativeAnalysis(
                common_findings=["Error generating common findings"],
                contradictions=["Error generating contradictions"],
                research_gaps=["Error generating research gaps"],
                future_directions=["Error generating future directions"]
            )
    
    def __call__(self, paper_summaries: List[PaperSummary]) -> ComparativeAnalysis:
        """Process the paper summaries and return a comparative analysis."""
        return self.process(paper_summaries)

## 3. Agent Orchestration with LangGraph

Orchestrate the agents using LangGraph to create a complete research workflow.

In [ ]:
from typing_extensions import TypedDict

# Define the state for the LangGraph workflow
class ResearchState(TypedDict):
    query: str
    top_n: int
    keyword_set: Optional[KeywordSet]
    papers: Optional[List[Paper]]
    ranked_papers: Optional[List[RankedPaper]]
    paper_summaries: Optional[List[PaperSummary]]
    comparative_analysis: Optional[ComparativeAnalysis]
    research_report: Optional[ResearchReport]
    status_message: str # To track progress

# Initialize agents
keyword_agent = KeywordAgent(llm)
search_agent = SearchAgent()
rank_agent = RankAgent(llm)
summary_agent = SummaryAgent(model=model, tokenizer=tokenizer, llm=llm)
compare_agent = CompareAgent(llm)

# Define node functions for the graph
def run_keyword_expansion(state: ResearchState) -> ResearchState:
    """Expand keywords from the query."""
    print("\nExpanding keywords...")
    state["status_message"] = "Expanding keywords..."
    keyword_set = keyword_agent.process(state["query"])
    state["keyword_set"] = keyword_set
    print(f"Expanded keywords: {[k.keyword for k in keyword_set.expanded_keywords]}")
    return state

def run_literature_search(state: ResearchState) -> ResearchState:
    """Search for papers using the expanded keywords."""
    print("\nSearching for papers...")
    state["status_message"] = "Searching for papers..."
    papers = search_agent.process(state["keyword_set"])
    state["papers"] = papers
    print(f"Found {len(papers)} papers")
    return state

def run_paper_ranking(state: ResearchState) -> ResearchState:
    """Rank the papers based on multiple criteria."""
    print("\nRanking papers...")
    state["status_message"] = "Ranking papers..."
    ranked_papers = rank_agent.process(state["papers"], state["keyword_set"])
    state["ranked_papers"] = ranked_papers
    print("\nTop 5 ranked papers:")
    for i, paper in enumerate(ranked_papers[:5]):
        print(f"{i+1}. {paper.title} (Score: {paper.rank_score:.2f})")
    return state

def run_paper_summarization(state: ResearchState) -> ResearchState:
    """Generate summaries for the top-ranked papers."""
    print("\nSummarizing papers...")
    state["status_message"] = "Summarizing papers..."
    paper_summaries = summary_agent.process(state["ranked_papers"], state["top_n"])
    state["paper_summaries"] = paper_summaries
    return state

def run_comparative_analysis(state: ResearchState) -> ResearchState:
    """Perform comparative analysis on the paper summaries."""
    print("\nPerforming comparative analysis...")
    state["status_message"] = "Performing comparative analysis..."
    comparative_analysis = compare_agent.process(state["paper_summaries"])
    state["comparative_analysis"] = comparative_analysis
    return state

# Define topic summary generation (for the final report)
topic_summary_prompt = PromptTemplate(
    input_variables=["query", "keywords"],
    template="""You are an expert academic researcher writing a brief overview of a research topic.

Research topic: {query}

Related keywords: {keywords}

Write a concise (150-200 words) but comprehensive overview of this research topic that explains its significance, key aspects, and relevance in its field.
"""
)
topic_summary_chain = topic_summary_prompt | llm | StrOutputParser()

def generate_research_report(state: ResearchState) -> ResearchState:
    """Generate the final research report."""
    print("\nGenerating research report...")
    state["status_message"] = "Generating research report..."
    
    # Generate topic summary
    keywords_str = ", ".join([k.keyword for k in state["keyword_set"].expanded_keywords])
    topic_summary = topic_summary_chain.invoke({"query": state["query"], "keywords": keywords_str})
    
    # Create the research report
    research_report = ResearchReport(
        topic=state["query"],
        topic_summary=topic_summary,
        expanded_keywords=[k.keyword for k in state["keyword_set"].expanded_keywords],
        paper_summaries=state["paper_summaries"],
        comparative_analysis=state["comparative_analysis"],
        timestamp=datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    )
    
    state["research_report"] = research_report
    state["status_message"] = "Research complete!"
    return state

# Build the LangGraph workflow
def build_research_graph():
    """Build and return a compiled StateGraph for the research workflow."""
    # Create a new StateGraph with the ResearchState schema
    workflow = StateGraph(ResearchState)
    
    # Add nodes
    workflow.add_node("expand_keywords", run_keyword_expansion)
    workflow.add_node("search_papers", run_literature_search)
    workflow.add_node("rank_papers", run_paper_ranking)
    workflow.add_node("summarize_papers", run_paper_summarization)
    workflow.add_node("compare_papers", run_comparative_analysis)
    workflow.add_node("generate_report", generate_research_report)
    
    # Define edges
    workflow.set_entry_point("expand_keywords")
    workflow.add_edge("expand_keywords", "search_papers")
    workflow.add_edge("search_papers", "rank_papers")
    workflow.add_edge("rank_papers", "summarize_papers")
    workflow.add_edge("summarize_papers", "compare_papers")
    workflow.add_edge("compare_papers", "generate_report")
    workflow.add_edge("generate_report", END)
    
    # Compile the graph
    compiled_graph = workflow.compile()
    
    return compiled_graph

# Create the compiled research graph
research_graph = build_research_graph()

# Helper function to run the research workflow
def run_research(query: str, top_n: int = 3):
    """Run the complete research workflow with the graph."""
    # Initialize the state
    initial_state = {
        "query": query,
        "top_n": top_n,
        "status_message": "Starting research..."
    }
    
    # Invoke the graph
    print(f"Starting research workflow for query: {query}")
    final_state = research_graph.invoke(initial_state)
    
    # Return the research report
    return final_state["research_report"]

## 4. Testing and Evaluation

Test the multi-agent research assistant with a sample query.

In [ ]:
# Run the workflow with a sample query
sample_query = "Large Language Models in healthcare"
research_report = run_research(sample_query, top_n=3)

In [ ]:
# Display the research report
print(f"Research Report: {research_report.topic}")
print("="*80)
print("\nTopic Summary:")
print(research_report.topic_summary)

print("\nExpanded Keywords:")
for keyword in research_report.expanded_keywords:
    print(f"- {keyword}")

print("\nPaper Summaries:")
for i, summary in enumerate(research_report.paper_summaries):
    print(f"\nPaper {i+1}: {summary.title}")
    print(f"Authors: {', '.join(summary.authors)}")
    print(f"Summary: {summary.summary}")
    print(f"Methodology: {summary.methodology}")
    print(f"Key Contributions: {summary.contributions}")
    print(f"Limitations/Gaps: {summary.limitations}")

print("\nComparative Analysis:")
print("\nCommon Findings:")
for finding in research_report.comparative_analysis.common_findings:
    print(f"- {finding}")

print("\nContradictions:")
for contradiction in research_report.comparative_analysis.contradictions:
    print(f"- {contradiction}")

print("\nResearch Gaps:")
for gap in research_report.comparative_analysis.research_gaps:
    print(f"- {gap}")

print("\nFuture Directions:")
for direction in research_report.comparative_analysis.future_directions:
    print(f"- {direction}")

## 5. Research Report Generation

Create functions to export the research report in various formats.

In [ ]:
def export_report_to_markdown(research_report, output_file="research_report.md"):
    """Export the research report to a markdown file."""
    with open(output_file, "w") as f:
        f.write(f"# Research Report: {research_report.topic}\n\n")
        f.write(f"*Generated on: {research_report.timestamp}*\n\n")
        
        f.write("## Topic Summary\n\n")
        f.write(f"{research_report.topic_summary}\n\n")
        
        f.write("## Expanded Keywords\n\n")
        for keyword in research_report.expanded_keywords:
            f.write(f"- {keyword}\n")
        f.write("\n")
        
        f.write("## Paper Summaries\n\n")
        for i, summary in enumerate(research_report.paper_summaries):
            f.write(f"### Paper {i+1}: {summary.title}\n\n")
            f.write(f"**Authors:** {', '.join(summary.authors)}\n\n")
            f.write(f"**Summary:**\n{summary.summary}\n\n")
            f.write(f"**Methodology:**\n{summary.methodology}\n\n")
            f.write(f"**Key Contributions:**\n{summary.contributions}\n\n")
            f.write(f"**Limitations/Gaps:**\n{summary.limitations}\n\n")
        
        f.write("## Comparative Analysis\n\n")
        
        f.write("### Common Findings\n\n")
        for finding in research_report.comparative_analysis.common_findings:
            f.write(f"- {finding}\n")
        f.write("\n")
        
        f.write("### Contradictions\n\n")
        for contradiction in research_report.comparative_analysis.contradictions:
            f.write(f"- {contradiction}\n")
        f.write("\n")
        
        f.write("### Research Gaps\n\n")
        for gap in research_report.comparative_analysis.research_gaps:
            f.write(f"- {gap}\n")
        f.write("\n")
        
        f.write("### Future Directions\n\n")
        for direction in research_report.comparative_analysis.future_directions:
            f.write(f"- {direction}\n")
    
    print(f"Research report exported to {output_file}")
    return output_file

def export_report_to_json(research_report, output_file="research_report.json"):
    """Export the research report to a JSON file."""
    # Convert the Pydantic model to a dictionary
    report_dict = research_report.dict()
    
    with open(output_file, "w") as f:
        json.dump(report_dict, f, indent=2)
    
    print(f"Research report exported to {output_file}")
    return output_file

In [ ]:
# Export the research report to markdown and JSON
markdown_file = export_report_to_markdown(research_report)
json_file = export_report_to_json(research_report)

## 6. Creating a Streamlit Interface

Create a Streamlit interface for the multi-agent research assistant.

In [ ]:
%%writefile app_research_assistant.py
import streamlit as st
import os
import json
import time
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import together
from pathlib import Path
import sys
import torch
import importlib
import requests
import re
from urllib.parse import quote
import xml.etree.ElementTree as ET
import feedparser
import arxiv
from typing import List, Dict, Any, Optional
from pydantic import BaseModel, Field
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_openai import ChatOpenAI
from langgraph.graph import END, StateGraph
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

# Import the agent classes and utility functions
sys.path.append(os.path.dirname(os.path.abspath(__file__)))

# Data models (copied from notebook to ensure consistency)
class Keyword(BaseModel):
    """A keyword or phrase for academic search."""
    keyword: str = Field(description="The keyword or phrase")
    relevance: Optional[float] = Field(default=1.0, description="Relevance score from 0.0 to 1.0")
    
class KeywordSet(BaseModel):
    """A set of keywords for academic search."""
    original_query: str = Field(description="Original user query")
    expanded_keywords: List[Keyword] = Field(description="List of expanded keywords")
    
class Author(BaseModel):
    """An author of an academic paper."""
    name: str = Field(description="Author's name")
    affiliation: Optional[str] = Field(default=None, description="Author's affiliation")
    
class Paper(BaseModel):
    """An academic paper."""
    id: str = Field(description="Unique identifier for the paper")
    title: str = Field(description="Title of the paper")
    authors: List[Author] = Field(description="List of authors")
    abstract: str = Field(description="Abstract of the paper")
    year: Optional[int] = Field(default=None, description="Year of publication")
    url: Optional[str] = Field(default=None, description="URL to the paper")
    pdf_url: Optional[str] = Field(default=None, description="URL to the PDF")
    citation_count: Optional[int] = Field(default=None, description="Number of citations")
    source: str = Field(description="Source of the paper (e.g., 'arxiv', 'semantic_scholar')")
    keywords: Optional[List[str]] = Field(default=None, description="Keywords associated with the paper")
    full_text: Optional[str] = Field(default=None, description="Full text of the paper if available")
    
class RankedPaper(Paper):
    """An academic paper with ranking information."""
    rank_score: float = Field(description="Overall ranking score")
    relevance_score: float = Field(description="Relevance to the query")
    citation_score: float = Field(description="Score based on citations")
    recency_score: float = Field(description="Score based on publication recency")
    
class PaperSummary(BaseModel):
    """A summary of an academic paper."""
    paper_id: str = Field(description="ID of the summarized paper")
    title: str = Field(description="Title of the paper")
    authors: List[str] = Field(description="List of author names")
    summary: str = Field(description="Generated summary of the paper")
    methodology: Optional[str] = Field(default=None, description="Summary of the methodology used")
    contributions: Optional[str] = Field(default=None, description="Key contributions of the paper")
    limitations: Optional[str] = Field(default=None, description="Limitations or gaps identified")
    
class ComparativeAnalysis(BaseModel):
    """Comparative analysis of multiple paper summaries."""
    common_findings: List[str] = Field(description="Common findings across papers")
    contradictions: List[str] = Field(description="Contradictory findings between papers")
    research_gaps: List[str] = Field(description="Identified research gaps")
    future_directions: List[str] = Field(description="Suggested future research directions")
    
class ResearchReport(BaseModel):
    """Complete research report."""
    topic: str = Field(description="Research topic")
    topic_summary: str = Field(description="Brief overview of the topic")
    expanded_keywords: List[str] = Field(description="Expanded keywords used for search")
    paper_summaries: List[PaperSummary] = Field(description="Summaries of top papers")
    comparative_analysis: ComparativeAnalysis = Field(description="Comparative analysis of papers")
    timestamp: str = Field(description="Timestamp of the report generation")

# Define ResearchState for the LangGraph workflow
class ResearchState(dict):
    """Dictionary-based state for the research workflow."""
    pass

# Agent implementations
class KeywordAgent:
    """Agent that expands user-provided keywords into a comprehensive set of search terms."""
    
    def __init__(self, llm):
        self.llm = llm
        
        # Define the prompt template
        self.prompt_template = PromptTemplate(
            input_variables=["query"],
            template="""You are an expert research assistant specialized in expanding search queries into comprehensive sets of keywords for academic research.

Given the following research query, generate an expanded list of 8-12 related keywords and phrases that would be useful for finding relevant academic papers. 
Include both broader and more specific terms. Consider synonyms, related concepts, and any important subtopics.

Research query: {query}

Please respond with a JSON object in the following format:
{{"original_query": "the original query",
"expanded_keywords": [
    {{"keyword": "first keyword", "relevance": 0.9}},
    {{"keyword": "second keyword", "relevance": 0.8}},
    ... and so on
]}}

Relevance should be a float between 0.0 and 1.0, where 1.0 indicates highest relevance to the original query.
Order the keywords by relevance, with the most relevant ones first.
"""
        )
        
        # Set up the output parser
        self.output_parser = JsonOutputParser()
        
        # Create the chain
        self.chain = self.prompt_template | self.llm | self.output_parser
    
    def process(self, query: str) -> KeywordSet:
        """Expand the user's query into a set of related keywords."""
        try:
            # Generate expanded keywords using the LLM
            result = self.chain.invoke({"query": query})
            
            # Convert the result to a KeywordSet object
            expanded_keywords = [Keyword(keyword=item["keyword"], relevance=item["relevance"]) 
                                for item in result["expanded_keywords"]]
            
            return KeywordSet(
                original_query=result["original_query"],
                expanded_keywords=expanded_keywords
            )
        except Exception as e:
            print(f"Error in KeywordAgent: {e}")
            # Fallback: return the original query as a keyword
            return KeywordSet(
                original_query=query,
                expanded_keywords=[Keyword(keyword=query, relevance=1.0)]
            )

class SearchAgent:
    """Agent that searches for academic papers across multiple sources."""
    
    def __init__(self):
        self.MAX_RESULTS_PER_SOURCE = 5  # Max results to retrieve from each source
        self.semantic_scholar_api_key = os.environ.get("SEMANTIC_SCHOLAR_API_KEY")
    
    def search_arxiv_api(self, query, max_results=5):
        """Search arXiv using the official API."""
        try:
            # Format the query for the arXiv API
            encoded_query = quote(query)
            url = f"http://export.arxiv.org/api/query?search_query=all:{encoded_query}&start=0&max_results={max_results}"
            
            # Send the request
            response = requests.get(url)
            response.raise_for_status()
            
            # Parse the response using feedparser
            feed = feedparser.parse(response.content)
            
            papers = []
            for entry in feed.entries:
                # Extract the year from the published date
                published = entry.get('published', '')
                year = int(published[:4]) if published and len(published) >= 4 else None
                
                # Extract authors
                authors = []
                for author in entry.get('authors', []):
                    name = author.get('name', '')
                    if name:
                        authors.append(Author(name=name))
                
                # Create the paper object
                paper = Paper(
                    id=entry.get('id', '').split('/')[-1],
                    title=entry.get('title', ''),
                    authors=authors,
                    abstract=entry.get('summary', ''),
                    year=year,
                    url=entry.get('link', ''),
                    pdf_url=next((link.href for link in entry.get('links', []) if link.get('title') == 'pdf'), None),
                    citation_count=None,
                    source="arxiv",
                    keywords=None,
                )
                papers.append(paper)
            
            return papers
        except Exception as e:
            print(f"Error searching arXiv API: {e}")
            return []
    
    def search_semantic_scholar(self, query, max_results=5):
        """Search Semantic Scholar for papers matching the query."""
        try:
            # Use the Semantic Scholar API
            url = "https://api.semanticscholar.org/graph/v1/paper/search"
            params = {
                "query": query,
                "limit": max_results,
                "fields": "title,authors,abstract,year,url,citationCount,influentialCitationCount"
            }
            
            headers = {}
            if self.semantic_scholar_api_key:
                headers["x-api-key"] = self.semantic_scholar_api_key
            
            response = requests.get(url, params=params, headers=headers)
            response.raise_for_status()
            data = response.json()
            
            papers = []
            for item in data.get('data', []):
                # Extract authors
                authors = []
                for author in item.get('authors', []):
                    name = author.get('name', '')
                    if name:
                        authors.append(Author(name=name))
                
                # Create the paper object
                paper = Paper(
                    id=item.get('paperId', ''),
                    title=item.get('title', ''),
                    authors=authors,
                    abstract=item.get('abstract', ''),
                    year=item.get('year'),
                    url=item.get('url'),
                    pdf_url=None,  # Semantic Scholar API doesn't directly provide PDF URL
                    citation_count=item.get('citationCount'),
                    source="semantic_scholar",
                    keywords=None,
                )
                papers.append(paper)
            
            return papers
        except Exception as e:
            print(f"Error searching Semantic Scholar: {e}")
            return []
    
    def search_pubmed(self, query, max_results=5):
        """Search PubMed for papers matching the query."""
        try:
            # Format the query for the PubMed eUtils API
            encoded_query = quote(query)
            
            # First get the IDs of matching articles
            esearch_url = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term={encoded_query}&retmode=json&retmax={max_results}"
            esearch_response = requests.get(esearch_url)
            esearch_response.raise_for_status()
            esearch_data = esearch_response.json()
            
            # Get the list of IDs
            id_list = esearch_data.get('esearchresult', {}).get('idlist', [])
            if not id_list:
                return []
            
            # Then fetch details for these IDs
            ids_string = ",".join(id_list)
            efetch_url = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id={ids_string}&retmode=xml"
            efetch_response = requests.get(efetch_url)
            efetch_response.raise_for_status()
            
            # Parse the XML response
            try:
                root = ET.fromstring(efetch_response.content)
                papers = []
                
                for article in root.findall(".//PubmedArticle"):
                    try:
                        # Extract PubMed ID
                        pmid = article.find(".//PMID").text if article.find(".//PMID") is not None else "unknown"
                        
                        # Extract article title
                        title_element = article.find(".//ArticleTitle")
                        title = title_element.text if title_element is not None else "Unknown Title"
                        
                        # Extract abstract
                        abstract_elements = article.findall(".//AbstractText")
                        abstract = " ".join([elem.text for elem in abstract_elements if elem.text is not None]) if abstract_elements else ""
                        
                        # Extract year
                        year_element = article.find(".//PubDate/Year")
                        year = int(year_element.text) if year_element is not None and year_element.text.isdigit() else None
                        
                        # Extract authors
                        authors = []
                        author_elements = article.findall(".//Author")
                        for author_elem in author_elements:
                            last_name = author_elem.find("LastName")
                            fore_name = author_elem.find("ForeName")
                            name = ""
                            if last_name is not None and last_name.text is not None:
                                name += last_name.text
                                if fore_name is not None and fore_name.text is not None:
                                    name = f"{fore_name.text} {name}"
                            if name:
                                authors.append(Author(name=name))
                        
                        # Create URL to the PubMed article
                        url = f"https://pubmed.ncbi.nlm.nih.gov/{pmid}/"
                        
                        # Create paper object
                        paper = Paper(
                            id=pmid,
                            title=title,
                            authors=authors,
                            abstract=abstract,
                            year=year,
                            url=url,
                            pdf_url=None,  # PubMed doesn't directly provide PDF URLs
                            citation_count=None,  # PubMed doesn't provide citation counts
                            source="pubmed",
                            keywords=None,  # We could extract keywords if needed
                        )
                        papers.append(paper)
                    except Exception as e:
                        print(f"Error processing PubMed article: {e}")
                        continue
                
                return papers
            except Exception as e:
                print(f"Error parsing PubMed XML: {e}")
                return []
                
        except Exception as e:
            print(f"Error searching PubMed: {e}")
            return []
    
    def process(self, keyword_set: KeywordSet) -> List[Paper]:
        """Search for papers using the expanded keywords."""
        all_papers = []
        
        # Use the top 3 most relevant keywords for searching
        top_keywords = sorted(keyword_set.expanded_keywords, key=lambda k: k.relevance, reverse=True)[:3]
        
        # Search using the original query first
        print(f"Searching for papers with query: {keyword_set.original_query}")
        
        # Search arXiv
        arxiv_papers = self.search_arxiv_api(keyword_set.original_query, self.MAX_RESULTS_PER_SOURCE)
        if arxiv_papers:
            all_papers.extend(arxiv_papers)
        
        # Search PubMed
        pubmed_papers = self.search_pubmed(keyword_set.original_query, self.MAX_RESULTS_PER_SOURCE)
        if pubmed_papers:
            all_papers.extend(pubmed_papers)
        
        # Search Semantic Scholar if API key is available
        if self.semantic_scholar_api_key:
            ss_papers = self.search_semantic_scholar(keyword_set.original_query, self.MAX_RESULTS_PER_SOURCE)
            if ss_papers:
                all_papers.extend(ss_papers)
        
        # Search using each of the top keywords
        for keyword in top_keywords:
            # Search arXiv with each keyword
            arxiv_papers = self.search_arxiv_api(keyword.keyword, self.MAX_RESULTS_PER_SOURCE)
            if arxiv_papers:
                all_papers.extend(arxiv_papers)
            
            # Keep the app responsive by not searching too many sources
            # In a production app, you might want to add more comprehensive searching
        
        # Remove duplicates based on paper ID
        unique_papers = []
        paper_ids = set()
        
        for paper in all_papers:
            if paper.id not in paper_ids:
                unique_papers.append(paper)
                paper_ids.add(paper.id)
        
        return unique_papers

def load_fine_tuned_model(model_path="./fine_tuned_model"):
    """
    Load the fine-tuned model for paper summarization.
    
    Args:
        model_path: Path to the fine-tuned model
        
    Returns:
        model, tokenizer: The loaded model and tokenizer
    """
    # Check if model path exists
    if not os.path.exists(model_path):
        print(f"Warning: Model path {model_path} does not exist.")
        print("Using fallback method with Together.ai API for summarization.")
        return None, None
    
    try:
        # Load the base model
        base_model_name = "mistralai/Mistral-7B-v0.1"
        
        # Load tokenizer
        tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
        tokenizer.pad_token = tokenizer.eos_token
        
        # Check if CUDA is available
        device = "cuda" if torch.cuda.is_available() else "cpu"
        
        # Load the base model
        base_model = AutoModelForCausalLM.from_pretrained(
            base_model_name,
            load_in_4bit=True if torch.cuda.is_available() else False,
            torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
            device_map="auto" if torch.cuda.is_available() else None,
            trust_remote_code=True
        )
        
        # Load the fine-tuned model
        model = PeftModel.from_pretrained(base_model, model_path)
        model.eval()  # Set model to evaluation mode
        
        print("Fine-tuned model loaded successfully.")
        return model, tokenizer
    except Exception as e:
        print(f"Error loading fine-tuned model: {e}")
        print("Using fallback method with Together.ai API for summarization.")
        return None, None

# Load LLM
def get_llm():
    if "OPENAI_API_KEY" in os.environ and os.environ["OPENAI_API_KEY"]:
        return ChatOpenAI(
            model="gpt-3.5-turbo", 
            temperature=0.3, 
            max_tokens=1024
        )
    elif "TOGETHER_API_KEY" in os.environ and os.environ["TOGETHER_API_KEY"]:
        from langchain_community.llms import Together
        return Together(
            model="meta-llama/Llama-3-8B-Instruct", 
            temperature=0.3, 
            max_tokens=1024
        )
    else:
        return None

# Define a function to build and return the research graph
def build_research_graph(model, tokenizer, llm):
    from typing_extensions import TypedDict
    
    # Import required agents
    from langchain_core.runnables import RunnablePassthrough
    
    # Define the state for the LangGraph workflow
    class ResearchState(TypedDict):
        query: str
        top_n: int
        keyword_set: Optional[KeywordSet]
        papers: Optional[List[Paper]]
        ranked_papers: Optional[List[RankedPaper]]
        paper_summaries: Optional[List[PaperSummary]]
        comparative_analysis: Optional[ComparativeAnalysis]
        research_report: Optional[ResearchReport]
        status_message: str # To track progress
    
    # Initialize agents
    keyword_agent = KeywordAgent(llm)
    search_agent = SearchAgent()
    
    # Define node functions for the graph
    def run_keyword_expansion(state: ResearchState) -> ResearchState:
        """Expand keywords from the query."""
        state["status_message"] = "Expanding keywords..."
        keyword_set = keyword_agent.process(state["query"])
        state["keyword_set"] = keyword_set
        return state

    def run_literature_search(state: ResearchState) -> ResearchState:
        """Search for papers using the expanded keywords."""
        state["status_message"] = "Searching for papers..."
        papers = search_agent.process(state["keyword_set"])
        state["papers"] = papers
        return state

    def run_paper_ranking(state: ResearchState) -> ResearchState:
        """Rank the papers based on multiple criteria."""
        state["status_message"] = "Ranking papers..."
        # Simple ranking based on recency
        from datetime import datetime
        current_year = datetime.now().year
        
        ranked_papers = []
        for paper in state["papers"]:
            # Simplified ranking mechanism for the app
            if paper.year is not None:
                recency_score = max(0.1, min(1.0, 1.0 - 0.1 * (current_year - paper.year)))
            else:
                recency_score = 0.5
                
            relevance_score = 0.8  # Assume high relevance since we searched for these terms
            citation_score = 0.6    # Placeholder without actual citation data
            
            # Calculate overall rank score
            rank_score = 0.6 * relevance_score + 0.25 * citation_score + 0.15 * recency_score
            
            ranked_paper = RankedPaper(
                **paper.dict(),
                rank_score=rank_score,
                relevance_score=relevance_score,
                citation_score=citation_score,
                recency_score=recency_score
            )
            ranked_papers.append(ranked_paper)
        
        # Sort papers by rank score in descending order
        ranked_papers.sort(key=lambda p: p.rank_score, reverse=True)
        state["ranked_papers"] = ranked_papers
        return state

    def run_paper_summarization(state: ResearchState) -> ResearchState:
        """Generate summaries for the top-ranked papers."""
        state["status_message"] = "Summarizing papers..."
        # Select the top N papers
        top_papers = state["ranked_papers"][:state["top_n"]]
        
        paper_summaries = []
        for paper in top_papers:
            # For the app, use Together API for summarization
            try:
                # Prepare the author string
                authors_str = ", ".join([author.name for author in paper.authors])
                
                # Create the prompt
                prompt = f"""Summarize the following academic paper with a focus on the main problem, methods, and key findings.
                
Title: {paper.title}
Authors: {authors_str}
Abstract: {paper.abstract}
                
Provide a concise but comprehensive summary of the paper that covers the research problem, methodology, and major contributions.
Summary:"""
                
                # Call the Together.ai API
                response = together.Complete.create(
                    prompt=prompt,
                    model="meta-llama/Llama-3-8B-Instruct",
                    max_tokens=512,
                    temperature=0.7,
                    top_p=0.9,
                    top_k=60,
                    repetition_penalty=1.1
                )
                
                # Extract the summary
                summary = response['output']['choices'][0]['text']
                
                # Process structured information
                structure_prompt = f"""You are an expert academic researcher and summarizer. Extract and structure the information from the following paper summary into specific sections.

Paper summary: {summary}

Extract the following components from the summary:
1. Methodology: What research methods or techniques were used?
2. Key Contributions: What are the main findings or contributions of the paper?
3. Limitations or Gaps: What limitations, open questions, or research gaps were identified?

Respond with a JSON object in the following format:
{{"methodology": "description of methodology",
"contributions": "key contributions and findings",
"limitations": "limitations and research gaps"}}

Include only the JSON in your response, with no additional text."""
                
                structure_response = together.Complete.create(
                    prompt=structure_prompt,
                    model="meta-llama/Llama-3-8B-Instruct",
                    max_tokens=512,
                    temperature=0.3,
                    top_p=0.9,
                    top_k=60,
                    repetition_penalty=1.1
                )
                
                # Extract and parse the structure
                structure_text = structure_response['output']['choices'][0]['text']
                try:
                    import json
                    # Clean up the JSON text (remove markdown code markers if present)
                    structure_text = structure_text.replace("```json", "").replace("```", "").strip()
                    structured_info = json.loads(structure_text)
                except:
                    structured_info = {
                        "methodology": "Not available",
                        "contributions": "Not available",
                        "limitations": "Not available"
                    }
                
                # Create PaperSummary
                paper_summary = PaperSummary(
                    paper_id=paper.id,
                    title=paper.title,
                    authors=[author.name for author in paper.authors],
                    summary=summary,
                    methodology=structured_info.get("methodology", "Not available"),
                    contributions=structured_info.get("contributions", "Not available"),
                    limitations=structured_info.get("limitations", "Not available")
                )
                paper_summaries.append(paper_summary)
                
            except Exception as e:
                # Fallback for errors
                print(f"Error summarizing paper: {e}")
                paper_summary = PaperSummary(
                    paper_id=paper.id,
                    title=paper.title,
                    authors=[author.name for author in paper.authors],
                    summary=f"Summary generation failed: {str(e)}",
                    methodology="Not available due to error",
                    contributions="Not available due to error",
                    limitations="Not available due to error"
                )
                paper_summaries.append(paper_summary)
        
        state["paper_summaries"] = paper_summaries
        return state

    def run_comparative_analysis(state: ResearchState) -> ResearchState:
        """Perform comparative analysis on the paper summaries."""
        state["status_message"] = "Performing comparative analysis..."
        try:
            # Format the paper summaries for the prompt
            formatted_summaries = ""
            for i, summary in enumerate(state["paper_summaries"]):
                formatted_summaries += f"Paper {i+1}: {summary.title}\n"
                formatted_summaries += f"Authors: {', '.join(summary.authors)}\n"
                formatted_summaries += f"Summary: {summary.summary}\n"
                formatted_summaries += f"Methodology: {summary.methodology}\n"
                formatted_summaries += f"Key Contributions: {summary.contributions}\n"
                formatted_summaries += f"Limitations/Gaps: {summary.limitations}\n\n"
            
            prompt = f"""You are an expert academic researcher analyzing multiple papers on a related topic. Perform a comparative analysis of the following paper summaries to identify common themes, contradictions, research gaps, and future directions.

Paper Summaries:
{formatted_summaries}

Based on these summaries, provide a comprehensive comparative analysis in the following JSON format:
{{"common_findings": [
    "Common finding 1",
    "Common finding 2",
    ... (at least 3-5 items)
],
"contradictions": [
    "Contradiction 1",
    "Contradiction 2",
    ... (identify any contradictions between papers)
],
"research_gaps": [
    "Research gap 1",
    "Research gap 2",
    ... (identify areas that need further research, at least 3-4 items)
],
"future_directions": [
    "Future direction 1",
    "Future direction 2",
    ... (suggest future research directions, at least 3-4 items)
]}}

Include only the JSON in your response, with no additional text."""

            # Get comparative analysis
            analysis_response = together.Complete.create(
                prompt=prompt,
                model="meta-llama/Llama-3-8B-Instruct",
                max_tokens=1024,
                temperature=0.3,
                top_p=0.9,
                top_k=60,
                repetition_penalty=1.1
            )
            
            analysis_text = analysis_response['output']['choices'][0]['text']
            
            # Parse JSON response
            import json
            # Clean up the JSON text (remove markdown code markers if present)
            analysis_text = analysis_text.replace("```json", "").replace("```", "").strip()
            analysis_data = json.loads(analysis_text)
            
            comparative_analysis = ComparativeAnalysis(
                common_findings=analysis_data.get("common_findings", ["No common findings identified"]),
                contradictions=analysis_data.get("contradictions", ["No contradictions identified"]),
                research_gaps=analysis_data.get("research_gaps", ["No research gaps identified"]),
                future_directions=analysis_data.get("future_directions", ["No future directions identified"])
            )
            
        except Exception as e:
            print(f"Error in comparative analysis: {e}")
            # Return a default ComparativeAnalysis object on error
            comparative_analysis = ComparativeAnalysis(
                common_findings=["Error generating common findings"],
                contradictions=["Error generating contradictions"],
                research_gaps=["Error generating research gaps"],
                future_directions=["Error generating future directions"]
            )
        
        state["comparative_analysis"] = comparative_analysis
        return state

    def generate_research_report(state: ResearchState) -> ResearchState:
        """Generate the final research report."""
        state["status_message"] = "Generating research report..."
        try:
            # Generate topic summary
            keywords_str = ", ".join([k.keyword for k in state["keyword_set"].expanded_keywords])
            
            prompt = f"""You are an expert academic researcher writing a brief overview of a research topic.

Research topic: {state["query"]}

Related keywords: {keywords_str}

Write a concise (150-200 words) but comprehensive overview of this research topic that explains its significance, key aspects, and relevance in its field."""
            
            summary_response = together.Complete.create(
                prompt=prompt,
                model="meta-llama/Llama-3-8B-Instruct",
                max_tokens=400,
                temperature=0.3,
                top_p=0.9
            )
            
            topic_summary = summary_response['output']['choices'][0]['text']
            
            # Create the research report
            research_report = ResearchReport(
                topic=state["query"],
                topic_summary=topic_summary,
                expanded_keywords=[k.keyword for k in state["keyword_set"].expanded_keywords],
                paper_summaries=state["paper_summaries"],
                comparative_analysis=state["comparative_analysis"],
                timestamp=datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            )
            
            state["research_report"] = research_report
            state["status_message"] = "Research complete!"
            
        except Exception as e:
            print(f"Error generating research report: {e}")
            state["status_message"] = f"Error generating report: {str(e)}"
            
        return state
    
    # Build the LangGraph workflow
    workflow = StateGraph(ResearchState)
    
    # Add nodes
    workflow.add_node("expand_keywords", run_keyword_expansion)
    workflow.add_node("search_papers", run_literature_search)
    workflow.add_node("rank_papers", run_paper_ranking)
    workflow.add_node("summarize_papers", run_paper_summarization)
    workflow.add_node("compare_papers", run_comparative_analysis)
    workflow.add_node("generate_report", generate_research_report)
    
    # Define edges
    workflow.set_entry_point("expand_keywords")
    workflow.add_edge("expand_keywords", "search_papers")
    workflow.add_edge("search_papers", "rank_papers")
    workflow.add_edge("rank_papers", "summarize_papers")
    workflow.add_edge("summarize_papers", "compare_papers")
    workflow.add_edge("compare_papers", "generate_report")
    workflow.add_edge("generate_report", END)
    
    # Compile the graph
    compiled_graph = workflow.compile()
    
    return compiled_graph

# Define a function to run the research workflow
def run_research(query, top_n=3, graph=None, llm=None):
    if graph is None or llm is None:
        return None
        
    # Initialize the state
    initial_state = {
        "query": query,
        "top_n": top_n,
        "status_message": "Starting research..."
    }
    
    # Invoke the graph
    try:
        final_state = graph.invoke(initial_state)
        return final_state["research_report"]
    except Exception as e:
        print(f"Error running research workflow: {e}")
        return None

# Export functions
def export_report_to_markdown(research_report, output_file="research_report.md"):
    """Export the research report to a markdown file."""
    with open(output_file, "w") as f:
        f.write(f"# Research Report: {research_report.topic}\n\n")
        f.write(f"*Generated on: {research_report.timestamp}*\n\n")
        
        f.write("## Topic Summary\n\n")
        f.write(f"{research_report.topic_summary}\n\n")
        
        f.write("## Expanded Keywords\n\n")
        for keyword in research_report.expanded_keywords:
            f.write(f"- {keyword}\n")
        f.write("\n")
        
        f.write("## Paper Summaries\n\n")
        for i, summary in enumerate(research_report.paper_summaries):
            f.write(f"### Paper {i+1}: {summary.title}\n\n")
            f.write(f"**Authors:** {', '.join(summary.authors)}\n\n")
            f.write(f"**Summary:**\n{summary.summary}\n\n")
            f.write(f"**Methodology:**\n{summary.methodology}\n\n")
            f.write(f"**Key Contributions:**\n{summary.contributions}\n\n")
            f.write(f"**Limitations/Gaps:**\n{summary.limitations}\n\n")
        
        f.write("## Comparative Analysis\n\n")
        
        f.write("### Common Findings\n\n")
        for finding in research_report.comparative_analysis.common_findings:
            f.write(f"- {finding}\n")
        f.write("\n")
        
        f.write("### Contradictions\n\n")
        for contradiction in research_report.comparative_analysis.contradictions:
            f.write(f"- {contradiction}\n")
        f.write("\n")
        
        f.write("### Research Gaps\n\n")
        for gap in research_report.comparative_analysis.research_gaps:
            f.write(f"- {gap}\n")
        f.write("\n")
        
        f.write("### Future Directions\n\n")
        for direction in research_report.comparative_analysis.future_directions:
            f.write(f"- {direction}\n")
    
    return output_file

def export_report_to_json(research_report, output_file="research_report.json"):
    """Export the research report to a JSON file."""
    # Convert the Pydantic model to a dictionary
    report_dict = research_report.dict()
    
    with open(output_file, "w") as f:
        json.dump(report_dict, f, indent=2)
    
    return output_file

# Main Streamlit app
def main():
    st.set_page_config(
        page_title="Multi-Agent Research Assistant",
        page_icon="🔍",
        layout="wide"
    )

    st.title("🔍 Multi-Agent Autonomous Research Assistant")
    
    st.markdown("""
    This application uses a multi-agent system to automate the academic research process. 
    Enter a research topic, and the system will:
    1. Expand your query into relevant keywords
    2. Search for academic papers across multiple sources (arXiv and PubMed)
    3. Rank the papers based on relevance, citations, and recency
    4. Generate summaries of the top papers using a fine-tuned model
    5. Perform comparative analysis to identify common themes and research gaps
    6. Generate a comprehensive research report
    """)
    
    # Initialize session state variables
    if 'api_key_set' not in st.session_state:
        st.session_state.api_key_set = False
    if 'research_status' not in st.session_state:
        st.session_state.research_status = ""
    if 'report' not in st.session_state:
        st.session_state.report = None
    if 'research_graph' not in st.session_state:
        st.session_state.research_graph = None
    if 'llm' not in st.session_state:
        st.session_state.llm = None
    if 'model' not in st.session_state:
        st.session_state.model = None
    if 'tokenizer' not in st.session_state:
        st.session_state.tokenizer = None
    if 'progress' not in st.session_state:
        st.session_state.progress = 0
        
    # Create a sidebar for API key input
    with st.sidebar:
        st.header("API Configuration")
        api_key = st.text_input("Enter Together.ai API Key", type="password")
        openai_api_key = st.text_input("Enter OpenAI API Key (optional)", type="password")
        
        if st.button("Save API Keys"):
            if api_key:
                os.environ["TOGETHER_API_KEY"] = api_key
                together.api_key = api_key
                st.session_state.api_key_set = True
                st.success("Together.ai API key saved!")
                
                # Load the LLM and prepare the graph
                st.session_state.llm = get_llm()
                
                # Load the fine-tuned model
                with st.spinner("Loading model..."):
                    st.session_state.model, st.session_state.tokenizer = load_fine_tuned_model()
                
                # Build the research graph
                with st.spinner("Preparing research graph..."):
                    st.session_state.research_graph = build_research_graph(
                        st.session_state.model,
                        st.session_state.tokenizer,
                        st.session_state.llm
                    )
            
            if openai_api_key:
                os.environ["OPENAI_API_KEY"] = openai_api_key
                st.success("OpenAI API key saved!")
                
                # Update LLM and graph if OpenAI API is preferred
                st.session_state.llm = get_llm()
                
                # Rebuild the research graph with OpenAI
                with st.spinner("Preparing research graph..."):
                    st.session_state.research_graph = build_research_graph(
                        st.session_state.model,
                        st.session_state.tokenizer,
                        st.session_state.llm
                    )
        
        st.divider()
        st.header("About")
        st.markdown("""
        This research assistant uses a multi-agent system consisting of:
        - **KeywordAgent**: Expands research queries
        - **SearchAgent**: Searches academic repositories (arXiv, PubMed)
        - **RankAgent**: Ranks papers by relevance
        - **SummaryAgent**: Summarizes papers
        - **CompareAgent**: Performs comparative analysis
        
        The agents are orchestrated using LangGraph.
        """)
    
    # Main content area
    col1, col2 = st.columns([2, 1])
    
    with col1:
        st.header("Enter Research Topic")
        query = st.text_input("Research Topic", placeholder="e.g., Large Language Models in healthcare")
        top_n = st.slider("Number of top papers to analyze", min_value=1, max_value=5, value=3)
        
        if st.button("Start Research", disabled=not st.session_state.api_key_set):
            if not query:
                st.error("Please enter a research topic.")
            elif st.session_state.research_graph is None:
                st.error("Research graph is not ready. Please check API configuration.")
            else:
                # Reset progress
                st.session_state.progress = 0
                st.session_state.research_status = "Starting research workflow..."
                st.session_state.report = None
                
                # Create a placeholder for the progress bar
                progress_placeholder = st.empty()
                status_placeholder = st.empty()
                
                # Start the research in a background thread to keep UI responsive
                import threading
                
                def update_progress():
                    progress = 0
                    stages = ["Expanding keywords...", "Searching for papers...", 
                              "Ranking papers...", "Summarizing papers...", 
                              "Performing comparative analysis...", "Generating report..."]
                    while progress < len(stages) and st.session_state.report is None:
                        progress_bar = progress_placeholder.progress(progress / len(stages))
                        status_placeholder.info(stages[progress])
                        st.session_state.progress = progress / len(stages)
                        st.session_state.research_status = stages[progress]
                        time.sleep(2)
                        progress += 1
                
                def do_research():
                    try:
                        report = run_research(
                            query=query, 
                            top_n=top_n,
                            graph=st.session_state.research_graph,
                            llm=st.session_state.llm
                        )
                        st.session_state.report = report
                        st.session_state.progress = 1.0
                        st.session_state.research_status = "Research complete!"
                    except Exception as e:
                        st.session_state.research_status = f"Error: {str(e)}"
                        print(f"Research error: {e}")
                
                # Start progress thread
                progress_thread = threading.Thread(target=update_progress)
                progress_thread.daemon = True
                progress_thread.start()
                
                # Start research thread
                research_thread = threading.Thread(target=do_research)
                research_thread.daemon = True
                research_thread.start()
                
                # Wait for threads to complete (or timeout after 30 seconds)
                time_waited = 0
                while research_thread.is_alive() and time_waited < 180:
                    time.sleep(1)
                    time_waited += 1
                    # Force a rerun every 5 seconds to update the UI
                    if time_waited % 5 == 0:
                        st.rerun()
                
                # Force a rerun to show results or error
                st.rerun()
    
    with col2:
        st.header("Research Status")
        status_placeholder = st.empty()
        
        if st.session_state.research_status:
            status_placeholder.info(st.session_state.research_status)
            # Show progress bar if ongoing
            if 0 < st.session_state.progress < 1:
                st.progress(st.session_state.progress)
        else:
            status_placeholder.info("Enter a research topic and click 'Start Research' to begin.")
    
    # Display research results
    if st.session_state.report is not None:
        display_research_report(st.session_state.report)

def display_research_report(report):
    """Display the research report in the Streamlit interface."""
    if isinstance(report, dict):
        # If the report is already a dictionary (from session state)
        report_dict = report
    else:
        # If the report is a Pydantic model
        report_dict = {
            "topic": report.topic,
            "topic_summary": report.topic_summary,
            "expanded_keywords": report.expanded_keywords,
            "paper_summaries": [summary.dict() for summary in report.paper_summaries],
            "comparative_analysis": report.comparative_analysis.dict(),
            "timestamp": report.timestamp
        }
    
    st.header(f"Research Report: {report_dict['topic']}")
    st.caption(f"Generated on: {report_dict['timestamp']}")
    
    st.subheader("Topic Summary")
    st.write(report_dict['topic_summary'])
    
    st.subheader("Expanded Keywords")
    # Display keywords as a horizontal list of tags
    keyword_html = ' '.join([f'<span style="background-color: #f0f2f6; padding: 5px 10px; margin-right: 10px; border-radius: 15px;">{keyword}</span>' for keyword in report_dict['expanded_keywords']])
    st.markdown(keyword_html, unsafe_allow_html=True)
    
    st.subheader("Paper Summaries")
    for i, summary in enumerate(report_dict['paper_summaries']):
        with st.expander(f"Paper {i+1}: {summary['title']}", expanded=i==0):
            st.markdown(f"**Authors:** {', '.join(summary['authors'])}")
            st.markdown(f"**Summary:**")
            st.write(summary['summary'])
            
            # Create three columns for methodology, contributions, and limitations
            col1, col2, col3 = st.columns(3)
            with col1:
                st.markdown(f"**Methodology:**")
                st.write(summary['methodology'])
            with col2:
                st.markdown(f"**Key Contributions:**")
                st.write(summary['contributions'])
            with col3:
                st.markdown(f"**Limitations/Gaps:**")
                st.write(summary['limitations'])
    
    st.subheader("Comparative Analysis")
    tab1, tab2, tab3, tab4 = st.tabs(["Common Findings", "Contradictions", "Research Gaps", "Future Directions"])
    
    with tab1:
        for finding in report_dict['comparative_analysis']['common_findings']:
            st.markdown(f"- {finding}")
    
    with tab2:
        for contradiction in report_dict['comparative_analysis']['contradictions']:
            st.markdown(f"- {contradiction}")
    
    with tab3:
        for gap in report_dict['comparative_analysis']['research_gaps']:
            st.markdown(f"- {gap}")
    
    with tab4:
        for direction in report_dict['comparative_analysis']['future_directions']:
            st.markdown(f"- {direction}")
    
    # Export options
    st.subheader("Export Report")
    col1, col2 = st.columns(2)
    
    # Create the report object for export functions
    if isinstance(report, dict):
        from pydantic import parse_obj_as
        report_obj = parse_obj_as(ResearchReport, report)
    else:
        report_obj = report
    
    with col1:
        if st.button("Export as Markdown"):
            try:
                output_file = export_report_to_markdown(report_obj)
                with open(output_file, 'r') as f:
                    md_content = f.read()
                st.download_button(
                    label="Download Markdown",
                    data=md_content,
                    file_name="research_report.md",
                    mime="text/markdown"
                )
                st.success(f"Report exported as {output_file}")
            except Exception as e:
                st.error(f"Error exporting to Markdown: {str(e)}")
    
    with col2:
        if st.button("Export as JSON"):
            try:
                output_file = export_report_to_json(report_obj)
                with open(output_file, 'r') as f:
                    json_content = f.read()
                st.download_button(
                    label="Download JSON",
                    data=json_content,
                    file_name="research_report.json",
                    mime="application/json"
                )
                st.success(f"Report exported as {output_file}")
            except Exception as e:
                st.error(f"Error exporting to JSON: {str(e)}")

if __name__ == "__main__":
    main()

To run the Streamlit app, execute the following command in the terminal:
```bash
streamlit run app_research_assistant.py
```

## 7. Summary and Conclusion

In this notebook, we have implemented a comprehensive Multi-Agent Autonomous Research Assistant that automates the academic research process. The system consists of five specialized agents:

1. **KeywordAgent**: Expands user queries into comprehensive search terms
2. **SearchAgent**: Retrieves relevant papers from academic repositories
3. **RankAgent**: Scores and ranks papers based on multiple criteria
4. **SummaryAgent**: Generates summaries of papers using a fine-tuned model
5. **CompareAgent**: Performs comparative analysis to identify patterns and gaps

These agents are orchestrated in a workflow that produces a comprehensive research report, making the literature review process more efficient and insightful. The system integrates with the fine-tuned summarization model from Part A to provide high-quality paper summaries.

The Multi-Agent Research Assistant demonstrates the potential of using LangChain/LangGraph for complex orchestration of specialized agents to automate knowledge work. Future improvements could include supporting more data sources, improving the ranking algorithm, and enhancing the user interface.